In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import nltk, re, os
from nltk.tokenize import RegexpTokenizer, word_tokenize, wordpunct_tokenize
from nltk.stem import WordNetLemmatizer, LancasterStemmer, PorterStemmer
from wordcloud import WordCloud
from nltk.corpus import stopwords

In [2]:
#

In [3]:
df = pd.read_csv('analyse_textuelles.csv')
df.head()

,text,Author,Tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational"


In [4]:
len(df['text'])

10

In [5]:
df.duplicated().sum()

0

In [6]:
df['Tags']

0          change, deep-thoughts, thinking, world
1                              abilities, choices
2    inspirational, life, live, miracle, miracles
3                aliteracy, books, classic, humor
4                      be-yourself, inspirational
5                       adulthood, success, value
6                                      life, love
7     edison, failure, inspirational, paraphrased
8                 misattributed-eleanor-roosevelt
9                          humor, obvious, simile
Name: Tags, dtype: object

In [7]:
def text_lower(text):
    return text.lower()

df['Tags'] = df['Tags'].apply(lambda x: text_lower(x))
df['text'] = df['text'].apply(lambda x: text_lower(x))
df['Author'] = df['Author'].apply(lambda x: text_lower(x))

In [8]:
## suppression des ponctuations et autres 
import time
%time
import string
def clean_text(text):
    #mettre tout le contenu du texte en miniscules
    #text = text.lower()
    #supprimer les ponctuations
    text = re.sub("\[.*?\]", '', text)
    #supprimer les urls html, asc
    text = re.sub("https?://\S+|www\.\S+", '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub("\n", '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


df['Tags'] = df['Tags'].apply(lambda x: clean_text(x))
df['text'] = df['text'].apply(lambda x: clean_text(x))
df['Author'] = df['Author'].apply(lambda x: clean_text(x))
df.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


,text,Author,Tags
0,“the world as we have created it is a process ...,albert einstein,change deepthoughts thinking world
1,“it is our choices harry that show what we tru...,jk rowling,abilities choices
2,“there are only two ways to live your life one...,albert einstein,inspirational life live miracle miracles
3,“the person be it gentleman or lady who has no...,jane austen,aliteracy books classic humor
4,“imperfection is beauty madness is genius and ...,marilyn monroe,beyourself inspirational


In [9]:
##tokenizer
tokenizer = RegexpTokenizer(r"\w+")
def token(text):
    return tokenizer.tokenize(text)

df['Tags'] = df['Tags'].apply(lambda x: token(x))
df['text'] = df['text'].apply(lambda x: token(x))
df['Author'] = df['Author'].apply(lambda x: token(x))

In [10]:
df.head()

,text,Author,Tags
0,"[the, world, as, we, have, created, it, is, a,...","[albert, einstein]","[change, deepthoughts, thinking, world]"
1,"[it, is, our, choices, harry, that, show, what...","[jk, rowling]","[abilities, choices]"
2,"[there, are, only, two, ways, to, live, your, ...","[albert, einstein]","[inspirational, life, live, miracle, miracles]"
3,"[the, person, be, it, gentleman, or, lady, who...","[jane, austen]","[aliteracy, books, classic, humor]"
4,"[imperfection, is, beauty, madness, is, genius...","[marilyn, monroe]","[beyourself, inspirational]"


In [11]:
## stopwords
def stop_words_text(text):
    stop_word = set(stopwords.words('english'))
    word_text = [word for word in text if word not in stop_word]
    return word_text

df['Tags'] = df['Tags'].apply(lambda x: stop_words_text(x))
df['text'] = df['text'].apply(lambda x: stop_words_text(x))
df['Author'] = df['Author'].apply(lambda x: stop_words_text(x))
df.head()

,text,Author,Tags
0,"[world, created, process, thinking, cannot, ch...","[albert, einstein]","[change, deepthoughts, thinking, world]"
1,"[choices, harry, show, truly, far, abilities]","[jk, rowling]","[abilities, choices]"
2,"[two, ways, live, life, one, though, nothing, ...","[albert, einstein]","[inspirational, life, live, miracle, miracles]"
3,"[person, gentleman, lady, pleasure, good, nove...","[jane, austen]","[aliteracy, books, classic, humor]"
4,"[imperfection, beauty, madness, genius, better...","[marilyn, monroe]","[beyourself, inspirational]"


In [12]:
##Leammatization or racinisation or normalisation
df_lema = df.copy()
lema= WordNetLemmatizer()
def lem_word(text):
    return [lema.lemmatize(word) for word in text]


df_lema['Tags'] = df_lema['Tags'].apply(lambda x: lem_word(x))
df_lema['text'] = df_lema['text'].apply(lambda x: lem_word(x))
df_lema['Author'] = df_lema['Author'].apply(lambda x: lem_word(x))

In [13]:
df_lema.head()

,text,Author,Tags
0,"[world, created, process, thinking, cannot, ch...","[albert, einstein]","[change, deepthoughts, thinking, world]"
1,"[choice, harry, show, truly, far, ability]","[jk, rowling]","[ability, choice]"
2,"[two, way, live, life, one, though, nothing, m...","[albert, einstein]","[inspirational, life, live, miracle, miracle]"
3,"[person, gentleman, lady, pleasure, good, nove...","[jane, austen]","[aliteracy, book, classic, humor]"
4,"[imperfection, beauty, madness, genius, better...","[marilyn, monroe]","[beyourself, inspirational]"


In [14]:
stemer = LancasterStemmer()
def stemm_word(text):
    return [stemer.stem(i) for i in text]

for col in df_lema.columns:
    df_lema[col] = df_lema[col].apply(lambda x: stemm_word(x))
df_lema.head()

,text,Author,Tags
0,"[world, cre, process, think, cannot, chang, wi...","[albert, einstein]","[chang, deepthought, think, world]"
1,"[cho, harry, show, tru, far, abl]","[jk, rowl]","[abl, cho]"
2,"[two, way, liv, lif, on, though, noth, mirac, ...","[albert, einstein]","[inspir, lif, liv, mirac, mirac]"
3,"[person, gentlem, lady, pleas, good, novel, mu...","[jan, aust]","[alit, book, class, hum]"
4,"[imperfect, beauty, mad, geni, bet, absolv, ri...","[marilyn, monro]","[beyourself, inspir]"


In [15]:
porter = PorterStemmer()
def porter_word(text):
    return [porter.stem(a) for a in text]

for col in df_lema.columns:
    df_lema[col] = df_lema[col].apply(lambda x: porter_word(x))

df_lema.head()

,text,Author,Tags
0,"[world, cre, process, think, cannot, chang, wi...","[albert, einstein]","[chang, deepthought, think, world]"
1,"[cho, harri, show, tru, far, abl]","[jk, rowl]","[abl, cho]"
2,"[two, way, liv, lif, on, though, noth, mirac, ...","[albert, einstein]","[inspir, lif, liv, mirac, mirac]"
3,"[person, gentlem, ladi, plea, good, novel, mus...","[jan, aust]","[alit, book, class, hum]"
4,"[imperfect, beauti, mad, geni, bet, absolv, ri...","[marilyn, monro]","[beyourself, inspir]"


In [16]:
## POST tagging
import nltk
from nltk.corpus import wordnet
from nltk.corpus import brown

word_net_map = {
    "N": wordnet.NOUN, 
    "V": wordnet.VERB,
    "R": wordnet.ADV,
    "J": wordnet.ADJ
}
train_sents = brown.tagged_sents(categories= "news")
t0 = nltk.DefaultTagger("NN")
t1 = nltk.UnigramTagger(train_sents, backoff =t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

#def post_tag_wordnet(text, post_tag_type = 'post_tag'):
    #pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    #pos_tagged_text = [(word, word_net_map.get(pos_tag[0])) if pos_tag[0] in word_net_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    #return pos_tagged_text

LookupError: 
**********************************************************************
  Resource [93mbrown[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('brown')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/brown[0m

  Searched in:
    - '/home/einstein/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


TEXT FEATURES EXTRACTION  ====== BAG OF WORDS / BAG OF N-GRAMS

In [17]:
df.shape

(10, 3)

In [18]:
df.head()

,text,Author,Tags
0,"[world, created, process, thinking, cannot, ch...","[albert, einstein]","[change, deepthoughts, thinking, world]"
1,"[choices, harry, show, truly, far, abilities]","[jk, rowling]","[abilities, choices]"
2,"[two, ways, live, life, one, though, nothing, ...","[albert, einstein]","[inspirational, life, live, miracle, miracles]"
3,"[person, gentleman, lady, pleasure, good, nove...","[jane, austen]","[aliteracy, books, classic, humor]"
4,"[imperfection, beauty, madness, genius, better...","[marilyn, monroe]","[beyourself, inspirational]"


In [19]:
lancaster = LancasterStemmer()
def stemming(text):
    return [lancaster.stem(word) for word in text]

for col in df.columns:
    df[col] = df[col].apply(lambda x: stemming(x))
df.head()

,text,Author,Tags
0,"[world, cre, process, think, cannot, chang, wi...","[albert, einstein]","[chang, deepthought, think, world]"
1,"[cho, harry, show, tru, far, abl]","[jk, rowl]","[abl, cho]"
2,"[two, way, liv, lif, on, though, noth, mirac, ...","[albert, einstein]","[inspir, lif, liv, mirac, mirac]"
3,"[person, gentlem, lady, pleas, good, novel, mu...","[jan, aust]","[alit, book, class, hum]"
4,"[imperfect, beauty, mad, geni, bet, absolv, ri...","[marilyn, monro]","[beyourself, inspir]"


In [20]:
def lematizer(text):

    """ WITHOUT POST TAGING Lemmatization of racinisation or normalisation . We can use  Lancastemmer , PorterStemmer, SnowBallStemmer and 
    also WordNetLemmatizer  """

    lemmatizer = WordNetLemmatizer()
    lem  = [lemmatizer.lemmatize(word) for word in text]
    return lem

for col in df.columns:
    df[col] = df[col].apply(lambda x: lematizer(x))

df.head()

,text,Author,Tags
0,"[world, cre, process, think, cannot, chang, wi...","[albert, einstein]","[chang, deepthought, think, world]"
1,"[cho, harry, show, tru, far, abl]","[jk, rowl]","[abl, cho]"
2,"[two, way, liv, lif, on, though, noth, mirac, ...","[albert, einstein]","[inspir, lif, liv, mirac, mirac]"
3,"[person, gentlem, lady, plea, good, novel, mus...","[jan, aust]","[alit, book, class, hum]"
4,"[imperfect, beauty, mad, geni, bet, absolv, ri...","[marilyn, monro]","[beyourself, inspir]"


In [21]:
##combined theirs text
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

for col in df.columns:
    df[col] = df[col].apply(lambda x: combine_text(x))
df.head()

,text,Author,Tags
0,world cre process think cannot chang without c...,albert einstein,chang deepthought think world
1,cho harry show tru far abl,jk rowl,abl cho
2,two way liv lif on though noth mirac though ev...,albert einstein,inspir lif liv mirac mirac
3,person gentlem lady plea good novel must intol...,jan aust,alit book class hum
4,imperfect beauty mad geni bet absolv ridic abs...,marilyn monro,beyourself inspir


In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def countor_vectorizer(data, ngram = 1, max_of_words = 100000):
    count_vectorize = CountVectorizer(ngram_range=(ngram, ngram), max_features= max_of_words)
    embedding = count_vectorize.fit_transform(data).toarray()
    print("countvectorizer with", str(np.array(embedding).shape[1]), "features")
    return embedding, count_vectorize

In [23]:
def print_out(embedding, feat, ngram, compared_sentence= 0):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names_out(), "\n")
    print(ngram,"bag-of-feature: ")
    print(test_count_1gram.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(embedding.transpose(), index = feat.get_feature_names_out()).head(), "\n")
    print(ngram,"vector example:")
    print(df['text'][compared_sentence])
    print(embedding[compared_sentence], "\n")

In [24]:
test_corpus = df['text'][:5].tolist()
print("The test corpus: ", test_corpus, "\n")

test_count_emb, test_count_1gram = countor_vectorizer(test_corpus, ngram=1)
print_out(test_count_emb, test_count_1gram, ngram="Uni-gram")

The test corpus:  ['world cre process think cannot chang without chang think', 'cho harry show tru far abl', 'two way liv lif on though noth mirac though everyth mirac', 'person gentlem lady plea good novel must intol stupid', 'imperfect beauty mad geni bet absolv ridic absolv bor'] 

countvectorizer with 39 features
Uni-gram bag-of-words: 
['abl' 'absolv' 'beauty' 'bet' 'bor' 'cannot' 'chang' 'cho' 'cre'
 'everyth' 'far' 'geni' 'gentlem' 'good' 'harry' 'imperfect' 'intol'
 'lady' 'lif' 'liv' 'mad' 'mirac' 'must' 'noth' 'novel' 'on' 'person'
 'plea' 'process' 'ridic' 'show' 'stupid' 'think' 'though' 'tru' 'two'
 'way' 'without' 'world'] 

Uni-gram bag-of-feature: 
{'world': 38, 'cre': 8, 'process': 28, 'think': 32, 'cannot': 5, 'chang': 6, 'without': 37, 'cho': 7, 'harry': 14, 'show': 30, 'tru': 34, 'far': 10, 'abl': 0, 'two': 35, 'way': 36, 'liv': 19, 'lif': 18, 'on': 25, 'though': 33, 'noth': 23, 'mirac': 21, 'everyth': 9, 'person': 26, 'gentlem': 12, 'lady': 17, 'plea': 27, 'good': 

In [25]:
test_corpus = df['text'][:5].tolist()
print("The test corpus: ", test_corpus, "\n")

test_count2_emb, test_count_2gram = countor_vectorizer(test_corpus, ngram=2)
print_out(test_count2_emb, test_count_2gram, ngram="bi-gram")

The test corpus:  ['world cre process think cannot chang without chang think', 'cho harry show tru far abl', 'two way liv lif on though noth mirac though everyth mirac', 'person gentlem lady plea good novel must intol stupid', 'imperfect beauty mad geni bet absolv ridic absolv bor'] 

countvectorizer with 39 features
bi-gram bag-of-words: 
['absolv bor' 'absolv ridic' 'beauty mad' 'bet absolv' 'cannot chang'
 'chang think' 'chang without' 'cho harry' 'cre process' 'everyth mirac'
 'far abl' 'geni bet' 'gentlem lady' 'good novel' 'harry show'
 'imperfect beauty' 'intol stupid' 'lady plea' 'lif on' 'liv lif'
 'mad geni' 'mirac though' 'must intol' 'noth mirac' 'novel must'
 'on though' 'person gentlem' 'plea good' 'process think' 'ridic absolv'
 'show tru' 'think cannot' 'though everyth' 'though noth' 'tru far'
 'two way' 'way liv' 'without chang' 'world cre'] 

bi-gram bag-of-feature: 
{'world': 38, 'cre': 8, 'process': 28, 'think': 32, 'cannot': 5, 'chang': 6, 'without': 37, 'cho': 7, 

In [26]:
test_corpus = df['text'][:5].tolist()
print("The test corpus: ", test_corpus, "\n")

test_count3_emb, test_count_3gram = countor_vectorizer(test_corpus, ngram=3)
print_out(test_count3_emb, test_count_3gram, ngram="Tri-gram")

The test corpus:  ['world cre process think cannot chang without chang think', 'cho harry show tru far abl', 'two way liv lif on though noth mirac though everyth mirac', 'person gentlem lady plea good novel must intol stupid', 'imperfect beauty mad geni bet absolv ridic absolv bor'] 

countvectorizer with 34 features
Tri-gram bag-of-words: 
['absolv ridic absolv' 'beauty mad geni' 'bet absolv ridic'
 'cannot chang without' 'chang without chang' 'cho harry show'
 'cre process think' 'geni bet absolv' 'gentlem lady plea'
 'good novel must' 'harry show tru' 'imperfect beauty mad'
 'lady plea good' 'lif on though' 'liv lif on' 'mad geni bet'
 'mirac though everyth' 'must intol stupid' 'noth mirac though'
 'novel must intol' 'on though noth' 'person gentlem lady'
 'plea good novel' 'process think cannot' 'ridic absolv bor'
 'show tru far' 'think cannot chang' 'though everyth mirac'
 'though noth mirac' 'tru far abl' 'two way liv' 'way liv lif'
 'without chang think' 'world cre process'] 

T

In [27]:
###TFIDF Vetorizer
def tifdf_vectorize(data, ngram= 1, max_of_word = 100000):
    tf_vectorizer = TfidfVectorizer(ngram_range=(ngram, ngram), max_features= max_of_word)
    embed = tf_vectorizer.fit_transform(data).toarray()
    print('the tifvectorize with', str(np.array(embed).shape[1]), "features")
    return embed, tf_vectorizer

In [28]:
text_corpus = df['Tags'][:5].tolist()
print(f"The text corpus is: {text_corpus}")

tff1_emb, tff1_gram = tifdf_vectorize(text_corpus, ngram=1)
print_out(tff1_emb, tff1_gram, ngram = 'Uni-gram')

The text corpus is: ['chang deepthought think world', 'abl cho', 'inspir lif liv mirac mirac', 'alit book class hum', 'beyourself inspir']
the tifvectorize with 15 features
Uni-gram bag-of-words: 
['abl' 'alit' 'beyourself' 'book' 'chang' 'cho' 'class' 'deepthought'
 'hum' 'inspir' 'lif' 'liv' 'mirac' 'think' 'world'] 

Uni-gram bag-of-feature: 
{'world': 38, 'cre': 8, 'process': 28, 'think': 32, 'cannot': 5, 'chang': 6, 'without': 37, 'cho': 7, 'harry': 14, 'show': 30, 'tru': 34, 'far': 10, 'abl': 0, 'two': 35, 'way': 36, 'liv': 19, 'lif': 18, 'on': 25, 'though': 33, 'noth': 23, 'mirac': 21, 'everyth': 9, 'person': 26, 'gentlem': 12, 'lady': 17, 'plea': 27, 'good': 13, 'novel': 24, 'must': 22, 'intol': 16, 'stupid': 31, 'imperfect': 15, 'beauty': 2, 'mad': 20, 'geni': 11, 'bet': 3, 'absolv': 1, 'ridic': 29, 'bor': 4} 

BoW matrix:
              0         1    2    3         4
abl         0.0  0.707107  0.0  0.0  0.000000
alit        0.0  0.000000  0.0  0.5  0.000000
beyourself  0.0  0

In [29]:
tff2_emb, tff2_gram = tifdf_vectorize(text_corpus, ngram=2)
print_out(tff2_emb, tff2_gram, ngram = 'bi_gram')

the tifvectorize with 12 features
bi_gram bag-of-words: 
['abl cho' 'alit book' 'beyourself inspir' 'book class'
 'chang deepthought' 'class hum' 'deepthought think' 'inspir lif'
 'lif liv' 'liv mirac' 'mirac mirac' 'think world'] 

bi_gram bag-of-feature: 
{'world': 38, 'cre': 8, 'process': 28, 'think': 32, 'cannot': 5, 'chang': 6, 'without': 37, 'cho': 7, 'harry': 14, 'show': 30, 'tru': 34, 'far': 10, 'abl': 0, 'two': 35, 'way': 36, 'liv': 19, 'lif': 18, 'on': 25, 'though': 33, 'noth': 23, 'mirac': 21, 'everyth': 9, 'person': 26, 'gentlem': 12, 'lady': 17, 'plea': 27, 'good': 13, 'novel': 24, 'must': 22, 'intol': 16, 'stupid': 31, 'imperfect': 15, 'beauty': 2, 'mad': 20, 'geni': 11, 'bet': 3, 'absolv': 1, 'ridic': 29, 'bor': 4} 

BoW matrix:
                         0    1    2        3    4
abl cho            0.00000  1.0  0.0  0.00000  0.0
alit book          0.00000  0.0  0.0  0.57735  0.0
beyourself inspir  0.00000  0.0  0.0  0.00000  1.0
book class         0.00000  0.0  0.0  0.57

BASIC WORD EMBEDDINGS METHODS


### WORD2Vec

In [30]:
%time
import gensim

print("gensim version :", gensim.__version__)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.3 µs
gensim version : 4.3.3


In [31]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv'